# 6 WorkFlow

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [3]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [4]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


ln: failed to create symbolic link '/content/buckets/b1/labo1': File exists
cp: cannot stat '/content/buckets/b1/kaggle/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Wed Nov 26 11:38:08 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660355,35.3,1454500,77.7,1454500,77.7
Vcells,1226516,9.4,8388608,64.0,1975136,15.1


In [56]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

# Placeholder for update_progress function
update_progress <- function(seed_idx, message_str) {
  message(paste("[Progress]", message_str))
}

# Placeholder for seed_log function
seed_log <- function(message_str, level = "INFO") {
  message(paste("[", level, "]", message_str))
}

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [57]:
PARAM <- list()
PARAM$semilla_primigenia <- 105727 #123457,150011,193939,409993

PARAM$experimento <- 1001
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [74]:
PARAM <- list()
PARAM$semilla_primigenia <- 105727 #123457,150011,193939,409993

PARAM$experimento <- 1001
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

# Define dataset_cache_file here
dataset_cache_file <- paste0("dataset_fe_cache.rds")

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [75]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 6.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [76]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [77]:
list.files("/content/buckets/b1/datasets")

[1] "consolidado_gridsearch_final.csv"  "consolidado_gridsearch_final.xlsx"
[3] "dataset_pequeno.csv"               "gerencial_competencia_2025.csv.gz"
[5] "Indice-FACPCE.csv"                 "Indice-FACPCE.xlsx"

In [78]:
# PASO 1
library(readxl)
library(dplyr)
library(data.table)

fapce <- read_excel(
  "/content/buckets/b1/datasets/Indice-FACPCE.xlsx",
 col_types = c("numeric", "numeric", "numeric") # forzar lectura como números
) %>%
  mutate(
    # ---- FOTO_MES ----
    foto_mes = as.Date(foto_mes, origin = "1899-12-30"),
    foto_mes = format(foto_mes, "%Y%m"),

    # ---- IPC ----
    IPC = as.numeric(IPC),

    # ---- coeficiente ajustes ----
    coeficiente_de_ajuste = as.numeric(coeficiente_de_ajuste)
  ) %>%
  as.data.table()

print(fapce[1:6, ])



Warning message:
“Expecting numeric in A2 / R2C1: got a date”
Warning message:
“Expecting numeric in A3 / R3C1: got a date”
Warning message:
“Expecting numeric in A4 / R4C1: got a date”
Warning message:
“Expecting numeric in A5 / R5C1: got a date”
Warning message:
“Expecting numeric in A6 / R6C1: got a date”
Warning message:
“Expecting numeric in A7 / R7C1: got a date”
Warning message:
“Expecting numeric in A8 / R8C1: got a date”
Warning message:
“Expecting numeric in A9 / R9C1: got a date”
Warning message:
“Expecting numeric in A10 / R10C1: got a date”
Warning message:
“Expecting numeric in A11 / R11C1: got a date”
Warning message:
“Expecting numeric in A12 / R12C1: got a date”
Warning message:
“Expecting numeric in A13 / R13C1: got a date”
Warning message:
“Expecting numeric in A14 / R14C1: got a date”
Warning message:
“Expecting numeric in A15 / R15C1: got a date”
Warning message:
“Expecting numeric in A16 / R16C1: got a date”
Warning message:
“Expecting numeric in A17 / R17C1: got 

   foto_mes   IPC coeficiente_de_ajuste
     <char> <num>                 <num>
1:   202005 314.9              1.678311
2:   202006 322.0              1.641304
3:   202007 328.2              1.610299
4:   202008 337.1              1.567784
5:   202009 346.6              1.524812
6:   202010 359.7              1.469280


In [79]:
head(fapce)

foto_mes,IPC,coeficiente_de_ajuste
<chr>,<dbl>,<dbl>
202005,314.9,1.678311
202006,322.0,1.641304
202007,328.2,1.610299
202008,337.1,1.567784
202009,346.6,1.524812
202010,359.7,1.469280


In [80]:
#PASO 2
dataset <- as.data.table(dataset)

# asegurar foto_mes como texto tipo "202005"
dataset[, foto_mes := as.character(foto_mes)]
dataset[, foto_mes := gsub("[^0-9]", "", foto_mes)]
dataset[, foto_mes := substr(foto_mes, 1, 6)]

In [81]:
#PASO 3
cols_borrar <- c("coeficiente_de_ajuste", "coeficiente_de_ajuste.x", "coeficiente_de_ajuste.y")

for (col in cols_borrar) {
  if (col %in% names(dataset)) dataset[, (col) := NULL]
}


In [82]:
#PASO 4
dataset <- merge(
  dataset,
  fapce[, .(foto_mes, coeficiente_de_ajuste)],
  by = "foto_mes",
  all.x = TRUE
)

In [83]:
#PASO 5 MERGE POR FOTO_MES
vars_ajustar <- c(
  "mrentabilidad",
  "mrentabilidad_annual",
  "mcomisiones",
  "mactivos_margen",
  "mpasivos_margen",
  "mcuenta_corriente",
  "mcaja_ahorro",
  "mcuentas_saldo",
  "mtarjeta_visa_consumo",
  "mtarjeta_master_consumo",
  "mprestamos_personales",
  "mpayroll",
  "Master_mpagominimo",
  "Visa_mpagominimo"
)

dataset[, (vars_ajustar) := lapply(.SD, as.numeric), .SDcols = vars_ajustar]

In [84]:
#PASO 6
dataset[
  ,
  (vars_ajustar) := lapply(.SD, function(x) x * coeficiente_de_ajuste),
  .SDcols = vars_ajustar
]

In [85]:
head(dataset)

foto_mes,numero_de_cliente,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria,coeficiente_de_ajuste
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<dbl>
202005,29186441,1,61,282,3952.3207,73445.539,2823.1534,-233.31874,1195.9641,⋯,20,47,0,6507,0.0000,0,7671,1752.10589,CONTINUA,1.678311
202005,29187961,1,61,312,8502.8416,79330.938,6504.4940,-5934.55654,7638.1760,⋯,228,232,0,5939,3386.0923,0,8090,12579.72656,CONTINUA,1.678311
202005,29193101,1,67,365,8071.7004,63770.196,11719.1896,-4971.55872,1862.0185,⋯,133,149,0,2026,4764.1531,0,2026,2775.80821,CONTINUA,1.678311
202005,29193281,1,54,158,5000.5599,148424.064,5622.3572,-1524.82907,994.7347,⋯,10,115,0,3125,1417.4340,0,3350,3405.77887,CONTINUA,1.678311
202005,29198891,1,54,312,5049.5330,37874.972,459.3200,-1080.61383,4993.8467,⋯,124,32,0,7202,393.7317,0,7242,2342.70338,CONTINUA,1.678311
202005,29205441,1,72,6,661.5732,2698.606,267.6738,-16.64884,352.4620,⋯,41,32,0,157,0.0000,0,157,98.43292,CONTINUA,1.678311


In [ ]:
# sin codigo en esta primera version del workflow

#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [86]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") )) {
  # Calcular los valores de kmes como un vector primero
  # Esto asegura que la longitud de `calculated_kmes` sea la misma que `nrow(dataset)`
  calculated_kmes <- as.integer(dataset$foto_mes) %% 100
  # Asignar el vector calculado a la nueva columna 'kmes'
  dataset[, kmes := calculated_kmes]
}

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

In [87]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "foto_mes"                    "numero_de_cliente"          
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "coeficiente_de_ajuste"       "kmes"                       
[35] "mpayroll_sobre_edad"

#### 6.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [ ]:
# No se implementa Feature Engineering a partir de Random Forest

#### 6.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [88]:
   # ======================================================================
      # FEATURE ENGINEERING HISTÓRICO AVANZADO
      # ======================================================================

      update_progress(seed_idx, "FEATURE_ENGINEERING")
      seed_log("Iniciando Feature Engineering Histórico AVANZADO...")

      inicio_fe <- Sys.time()
      cols_antes_fe <- ncol(dataset)

      # Variables base
      cols_lagueables <- setdiff(
        colnames(dataset),
        c("numero_de_cliente", "foto_mes", "clase_ternaria")
      )

      seed_log(paste("Variables base:", length(cols_lagueables)))

      # Ordenar por cliente y mes
      setorder(dataset, numero_de_cliente, foto_mes)

      # === TENDENCIAS (ventanas 3, 6) ===
      seed_log("Generando tendencias...")
      inicio_trends <- Sys.time()
      cols_antes_trends <- ncol(dataset)

      for (ventana in c(3, 6)) {
        for (col in cols_lagueables) {
          trend_col <- paste0(col, "_trend_", ventana)

          dataset[, (trend_col) := {
            if (.N >= ventana) {
              valores <- tail(get(col), ventana)
              if (all(is.na(valores))) {
                NA_real_
              } else {
                x <- 1:ventana
                y <- valores
                validos <- !is.na(y)
                if (sum(validos) >= 2) {
                  coef(lm(y[validos] ~ x[validos]))[2]
                } else {
                  NA_real_
                }
              }
            } else {
              NA_real_
            }
          }, by = numero_de_cliente]
        }
      }

      fin_trends <- Sys.time()
      cols_trends <- ncol(dataset) - cols_antes_trends
      seed_log(paste("Tendencias generadas:", cols_trends, "variables en",
                     round(as.numeric(difftime(fin_trends, inicio_trends, units = "mins")), 1), "min"),
               level = "SUCCESS")

      # Guardar dataset con FE en cache
      seed_log("Guardando dataset con FE en cache...", level = "INFO")
      inicio_save <- Sys.time()
      saveRDS(dataset, dataset_cache_file, compress = "xz")
      fin_save <- Sys.time()
      tiempo_save <- as.numeric(difftime(fin_save, inicio_save, units = "secs"))

      file_size_mb <- file.info(dataset_cache_file)$size / (1024^2)
      seed_log(paste("Dataset guardado en cache:", round(file_size_mb, 1), "MB en",
                     round(tiempo_save, 1), "seg"), level = "SUCCESS")



[Progress] FEATURE_ENGINEERING

[ INFO ] Iniciando Feature Engineering Histórico AVANZADO...

[ INFO ] Variables base: 32

[ INFO ] Generando tendencias...

[ SUCCESS ] Tendencias generadas: 64 variables en 16.2 min

[ INFO ] Guardando dataset con FE en cache...

[ SUCCESS ] Dataset guardado en cache: 20.2 MB en 33.6 seg



Verificacion de los campos recien creados

In [89]:
ncol(dataset)
colnames(dataset)

[1] 99

[1] "foto_mes"                            "numero_de_cliente"                  
 [3] "internet"                            "cliente_edad"                       
 [5] "cliente_antiguedad"                  "mrentabilidad"                      
 [7] "mrentabilidad_annual"                "mcomisiones"                        
 [9] "mactivos_margen"                     "mpasivos_margen"                    
[11] "cproductos"                          "mcuenta_corriente"                  
[13] "mcaja_ahorro"                        "cdescubierto_preacordado"           
[15] "mcuentas_saldo"                      "ctarjeta_visa_transacciones"        
[17] "mtarjeta_visa_consumo"               "mtarjeta_master_consumo"            
[19] "mprestamos_personales"               "cpayroll_trx"                       
[21] "mpayroll"                            "ccomisiones_mantenimiento"          
[23] "ccallcenter_transacciones"           "chomebanking_transacciones"         
[25] "ctrx_quarter"                        "Master_status"                      
[27] "Master_fechaalta"                    "Master_mpagominimo"                 
[29] "Visa_status"                         "Visa_fechaalta"                     
[31] "Visa_mpagominimo"                    "clase_ternaria"                     
[33] "coeficiente_de_ajuste"               "kmes"                               
[35] "mpayroll_sobre_edad"                 "internet_trend_3"                   
[37] "cliente_edad_trend_3"                "cliente_antiguedad_trend_3"         
[39] "mrentabilidad_trend_3"               "mrentabilidad_annual_trend_3"       
[41] "mcomisiones_trend_3"                 "mactivos_margen_trend_3"            
[43] "mpasivos_margen_trend_3"             "cproductos_trend_3"                 
[45] "mcuenta_corriente_trend_3"           "mcaja_ahorro_trend_3"               
[47] "cdescubierto_preacordado_trend_3"    "mcuentas_saldo_trend_3"             
[49] "ctarjeta_visa_transacciones_trend_3" "mtarjeta_visa_consumo_trend_3"      
[51] "mtarjeta_master_consumo_trend_3"     "mprestamos_personales_trend_3"      
[53] "cpayroll_trx_trend_3"                "mpayroll_trend_3"                   
[55] "ccomisiones_mantenimiento_trend_3"   "ccallcenter_transacciones_trend_3"  
[57] "chomebanking_transacciones_trend_3"  "ctrx_quarter_trend_3"               
[59] "Master_status_trend_3"               "Master_fechaalta_trend_3"           
[61] "Master_mpagominimo_trend_3"          "Visa_status_trend_3"                
[63] "Visa_fechaalta_trend_3"              "Visa_mpagominimo_trend_3"           
[65] "coeficiente_de_ajuste_trend_3"       "kmes_trend_3"                       
[67] "mpayroll_sobre_edad_trend_3"         "internet_trend_6"                   
[69] "cliente_edad_trend_6"                "cliente_antiguedad_trend_6"         
[71] "mrentabilidad_trend_6"               "mrentabilidad_annual_trend_6"       
[73] "mcomisiones_trend_6"                 "mactivos_margen_trend_6"            
[75] "mpasivos_margen_trend_6"             "cproductos_trend_6"                 
[77] "mcuenta_corriente_trend_6"           "mcaja_ahorro_trend_6"               
[79] "cdescubierto_preacordado_trend_6"    "mcuentas_saldo_trend_6"             
[81] "ctarjeta_visa_transacciones_trend_6" "mtarjeta_visa_consumo_trend_6"      
[83] "mtarjeta_master_consumo_trend_6"     "mprestamos_personales_trend_6"      
[85] "cpayroll_trx_trend_6"                "mpayroll_trend_6"                   
[87] "ccomisiones_mantenimiento_trend_6"   "ccallcenter_transacciones_trend_6"  
[89] "chomebanking_transacciones_trend_6"  "ctrx_quarter_trend_6"               
[91] "Master_status_trend_6"               "Master_fechaalta_trend_6"           
[93] "Master_mpagominimo_trend_6"          "Visa_status_trend_6"                
[95] "Visa_fechaalta_trend_6"              "Visa_mpagominimo_trend_6"           
[97] "coeficiente_de_ajuste_trend_6"       "kmes_trend_6"                       
[99] "mpayroll_sobre_edad_trend_

#### 6.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [90]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [91]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [92]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

####  6.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 10

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [ ]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

print(PARAM$out$lgbm$mejores_hiperparametros)

### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

##### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [ ]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "data-mining-analista-jr-2025-a"   o  la original "data-mining-analista-sr-2025-a"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")